# PySTAC-Client Introduction

This notebook shows basic use of pystac-client to open an API, iterate through Collections and Items, and perform simple spatio-temporal searches.

In [1]:
from pystac_client import Client

# set pystac_client logger to DEBUG to see API calls
import logging
logging.basicConfig()
logger = logging.getLogger('pystac_client')
logger.setLevel(logging.DEBUG)

# Client

We first connect to an API by retrieving the root catalog, or landing page, of the API with the `Client.open` function.

In [2]:
# STAC API root URL
URL = 'https://planetarycomputer.microsoft.com/api/stac/v1'

# custom headers
headers = []

cat = Client.open(URL, headers=headers)
cat

DEBUG:pystac_client.stac_api_io:GET https://planetarycomputer.microsoft.com/api/stac/v1 Headers: {'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


<Client id=microsoft-pc>

# CollectionClient

As with a static catalog the `get_collections` function will iterate through the Collections in the Catalog. Notice that because this is an API it can get all the Collections through a single call, rather than having to fetch each one individually.

In [3]:
for collection in cat.get_collections():
    print(collection)

DEBUG:pystac_client.stac_api_io:GET https://planetarycomputer.microsoft.com/api/stac/v1/ Headers: {'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
DEBUG:pystac_client.stac_api_io:GET https://planetarycomputer.microsoft.com/api/stac/v1/collections Headers: {'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


<CollectionClient id=daymet-annual-pr>
<CollectionClient id=daymet-daily-hi>
<CollectionClient id=3dep-seamless>
<CollectionClient id=3dep-lidar-dsm>
<CollectionClient id=fia>
<CollectionClient id=sentinel-1-rtc>
<CollectionClient id=gridmet>
<CollectionClient id=daymet-annual-na>
<CollectionClient id=daymet-monthly-na>
<CollectionClient id=daymet-annual-hi>
<CollectionClient id=daymet-monthly-hi>
<CollectionClient id=daymet-monthly-pr>
<CollectionClient id=gnatsgo-tables>
<CollectionClient id=hgb>
<CollectionClient id=cop-dem-glo-30>
<CollectionClient id=cop-dem-glo-90>
<CollectionClient id=goes-cmi>
<CollectionClient id=terraclimate>
<CollectionClient id=nasa-nex-gddp-cmip6>
<CollectionClient id=gpm-imerg-hhr>
<CollectionClient id=gnatsgo-rasters>
<CollectionClient id=3dep-lidar-hag>
<CollectionClient id=io-lulc-annual-v02>
<CollectionClient id=conus404>
<CollectionClient id=3dep-lidar-intensity>
<CollectionClient id=3dep-lidar-pointsourceid>
<CollectionClient id=mtbs>
<CollectionCli

In [4]:
collection = cat.get_collection('aster-l1t')
collection

DEBUG:pystac_client.stac_api_io:GET https://planetarycomputer.microsoft.com/api/stac/v1/collections/aster-l1t Headers: {'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


<CollectionClient id=aster-l1t>

# Items

The main functions for getting items return iterators, where pystac-client will handle retrieval of additional pages when needed. Note that one request is made for the first ten items, then a second request for the next ten.

In [5]:
items = collection.get_items()

# flush stdout so we can see the exact order that things happen
def get_ten_items(items):
    for i, item in enumerate(items):
        print(f"{i}: {item}", flush=True)
        if i == 9:
            return

print('First page', flush=True)
get_ten_items(items)

print('Second page', flush=True)
get_ten_items(items)

First page


DEBUG:pystac_client.stac_api_io:GET https://planetarycomputer.microsoft.com/api/stac/v1/collections/aster-l1t/items?collections=aster-l1t Headers: {'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


0: <Item id=AST_L1T_00312272006020322_20150518201805>
1: <Item id=AST_L1T_00312272006020313_20150518201753>
2: <Item id=AST_L1T_00312272006020304_20150518201753>
3: <Item id=AST_L1T_00312272006020256_20150518201805>
4: <Item id=AST_L1T_00312272006013731_20150518201743>
5: <Item id=AST_L1T_00312272006013722_20150518201738>
6: <Item id=AST_L1T_00312272006013714_20150518201738>
7: <Item id=AST_L1T_00312272006013705_20150517144401>
8: <Item id=AST_L1T_00312272006013656_20150517144401>
9: <Item id=AST_L1T_00312272006013647_20150517144406>
Second page


DEBUG:pystac_client.stac_api_io:GET https://planetarycomputer.microsoft.com/api/stac/v1/collections/aster-l1t/items?collections=aster-l1t&token=next:aster-l1t:AST_L1T_00312272006013647_20150517144406 Headers: {'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


0: <Item id=AST_L1T_00312272006013638_20150517144356>
1: <Item id=AST_L1T_00312272006013629_20150517144344>
2: <Item id=AST_L1T_00312272006013620_20150517144344>
3: <Item id=AST_L1T_00312272006013612_20150517144400>
4: <Item id=AST_L1T_00312272006013603_20150517144346>
5: <Item id=AST_L1T_00312272006013554_20150517144346>
6: <Item id=AST_L1T_00312272006013545_20150517144346>
7: <Item id=AST_L1T_00312272006013536_20150517144340>
8: <Item id=AST_L1T_00312272006013527_20150517144346>
9: <Item id=AST_L1T_00312272006013519_20150517144336>


# API Search

If the Catalog is an API, we have the ability to search for items based on spatio-temporal properties.

In [6]:
# AOI around Delfzijl, in northern Netherlands
geom = {
    "type": "Polygon",
    "coordinates": [
      [
        [
          6.42425537109375,
          53.174765470134616
        ],
        [
          7.344360351562499,
          53.174765470134616
        ],
        [
          7.344360351562499,
          53.67393435835391
        ],
        [
          6.42425537109375,
          53.67393435835391
        ],
        [
          6.42425537109375,
          53.174765470134616
        ]
      ]
    ]
}

# limit sets the # of items per page so we can see multiple pages getting fetched
search = cat.search(
    max_items = 15,
    limit = 5,
    collections = "aster-l1t",
    intersects = geom,
    datetime = "2000-01-01/2010-12-31",
)

items = list(search.items())

len(items)

DEBUG:pystac_client.stac_api_io:POST https://planetarycomputer.microsoft.com/api/stac/v1/search Headers: {'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Length': '353', 'Content-Type': 'application/json'} Payload: {"limit": 5, "datetime": "2000-01-01T00:00:00Z/2010-12-31T23:59:59Z", "collections": ["aster-l1t"], "intersects": {"type": "Polygon", "coordinates": [[[6.42425537109375, 53.174765470134616], [7.344360351562499, 53.174765470134616], [7.344360351562499, 53.67393435835391], [6.42425537109375, 53.67393435835391], [6.42425537109375, 53.174765470134616]]]}}
DEBUG:pystac_client.stac_api_io:POST https://planetarycomputer.microsoft.com/api/stac/v1/search Headers: {'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Length': '421', 'Content-Type': 'application/json'} Payload: {"limit": 5, "datetime": "2000-01-01T00:00:00Z/2010-

15

In [ ]:
# # note that this will work in JupyterLab, but not in a Jupyter Notebook

# import IPython.display


# IPython.display.JSON([i.to_dict() for i in items])

In [7]:
# this cell can be used in Jupyter Notebook. Use above if using JupyterLab

import json
import uuid
from IPython.display import display_javascript, display_html, display

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict) or isinstance(json_data, list):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;font: 12px/18px monospace !important;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
            renderjson.set_show_to_level(2);
            document.getElementById('%s').appendChild(renderjson(%s))
        });
      """ % (self.uuid, self.json_str), raw=True)


RenderJSON([i.to_dict() for i in items])